<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.62
    Uninstalling yfinance-0.2.62:
      Successfully uninstalled yfinance-0.2.62
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-06-10 13:17:51
-------------------
qualified stocks: 92
with latest results: 30
still star stocks: 17
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  1.01 L
Current:  1.35 C
-------------------
Today PnL: 38.70 K (0.29%)
Current PnL: -14.41 L (-10.04%)
CY Booked + Current PnL: -8.33 L (-5.81%)
-------------------
Total profit:  3.93 L
Total loss:  -18.34 L
-------------------
Total Booked + Current PnL: 18.65 L (16.0%)
Total Booked PnL: 33.06 L (28.37%)
Curr Year Booked PnL: 6.08 L (4.49%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.08 C
Est FTT PnL: 72.26 L (53.4%)
Deployed:  1.17 C
Current:  1.35 C
CAGR/XIRR %: 10.58%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,UJJIVANSFB,52.77,53.0,58.69,M-SC,90.22,137835.0,-4644.0,5265.0,5.11,-3.26,3.82,0.44,241.0,-0.88,1.03,62.68,OX40N,NTT,BANKS
4,ADANIPORTS,1103.69,1583.0,4.69,M-LC,58.70,105037.0,26675.0,7353.0,0.84,34.04,7.00,43.43,77.0,3.63,0.78,40.49,XY24,NTT,MISC
51,LICI,785.05,983.0,-4.34,H-LC,84.78,235377.0,40685.0,8403.0,-1.43,20.90,3.57,25.21,34.0,4.84,1.75,30.16,XY25,NTT,INSURANCE
61,RELIANCE,1291.83,1526.0,5.74,X-LC,23.91,157014.0,16205.0,9327.0,-0.57,11.51,5.94,18.13,19.0,1.74,1.17,23.98,XY25,NTT,REFINERIES
77,TATAELXSI,7332.28,7332.0,-11.28,X-MC,59.78,93233.0,-9419.0,9417.0,0.42,-9.18,10.10,-0.00,55.0,-1.00,0.69,40.14,OX40N,NTT,IT


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,AARTIIND,487.04,919.00,40.20,M-SC,38.04,80536.0,-800.0,72941.0,-0.29,-0.98,90.57,88.69,227.0,-0.01,0.60,35.81,XR,NTT,CHEMICALS
31,HAVELLS,1588.02,2069.16,-4.62,X-LC,21.74,246340.0,-1391.0,76439.0,2.29,-0.56,31.03,30.30,50.0,-0.02,1.83,12.66,X40,ATH,ELECTRICAL
6,APOLLOHOSP,6904.43,8285.00,-16.00,H-LC,19.57,103762.0,196.0,20514.0,-0.21,0.19,19.77,20.00,32.0,0.01,0.77,14.55,X40N,BTT,HEALTHCARE
35,HINDUNILVR,2413.81,2723.00,-14.69,X-LC,25.00,267938.0,-2409.0,37029.0,-0.04,-0.89,13.82,12.81,6.0,-0.07,2.00,10.89,XY25,NTT,FMCG


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,59.14,80.0,-53.72,L-SC,83.70,67162.0,-46387.0,86437.0,5.78,-40.85,128.70,35.27,260.0,-0.54,0.50,64.38,XR,NTT,HOTELS
83,UJJIVANSFB,52.77,53.0,58.69,M-SC,90.22,137835.0,-4644.0,5265.0,5.11,-3.26,3.82,0.44,241.0,-0.88,1.03,62.68,OX40N,NTT,BANKS
86,VALIANTORG,512.64,838.0,-381.97,H-SC,100.00,119769.0,-13517.0,98115.0,4.99,-10.14,81.92,63.47,144.0,-0.14,0.89,100.76,XR,NTT,CHEMICALS
88,VIPIND,488.80,489.0,-847.65,H-SC,52.17,71243.0,-24073.0,24109.0,3.48,-25.26,33.84,0.04,151.0,-1.00,0.53,40.52,OX40N,NTT,MISC
41,INDIGOPNTS,1407.73,1408.0,112.05,H-SC,77.17,139016.0,-35543.0,35574.0,3.36,-20.36,25.59,0.02,128.0,-1.00,1.04,20.55,OX40N,NTT,PAINTS


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,RBLBANK,230.61,388.0,44.99,M-SC,56.52,225664.0,-8636.0,168548.0,-3.55,-3.69,74.69,68.25,242.0,-0.05,1.68,48.61,XY24,NTT,BANKS
90,WHIRLPOOL,1167.49,2270.0,-33.40,M-SC,42.39,194231.0,22610.0,139458.0,-2.47,13.17,71.80,94.43,191.0,0.16,1.45,43.35,XR,NTT,DURABLES
73,STARHEALTH,551.56,761.0,1.24,M-SC,93.48,165383.0,-34833.0,110856.0,-2.38,-17.40,67.03,37.97,213.0,-0.31,1.23,32.79,XY24,NTT,INSURANCE
26,DMART,3764.92,5201.0,-20.19,X-LC,30.43,200528.0,16047.0,54323.0,-2.07,8.70,27.09,38.14,15.0,0.30,1.49,20.23,X40N,NTT,FMCG
11,BANDHANBNK,214.25,400.0,-29.09,H-SC,71.74,211897.0,-36633.0,252094.0,-1.92,-14.74,118.97,86.70,129.0,-0.15,1.58,40.18,XY24,NTT,BANKS


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,UJJIVANSFB,52.77,53.0,58.69,M-SC,90.22,137835.0,-4644.0,5265.0,5.11,-3.26,3.82,0.44,241.0,-0.88,1.03,62.68,OX40N,NTT,BANKS
77,TATAELXSI,7332.28,7332.0,-11.28,X-MC,59.78,93233.0,-9419.0,9417.0,0.42,-9.18,10.10,-0.00,55.0,-1.00,0.69,40.14,OX40N,NTT,IT
75,SYMPHONY,1306.42,1306.0,-27.66,M-SC,3.26,154318.0,-16823.0,16774.0,-0.11,-9.83,10.87,-0.03,189.0,-1.00,1.15,10.91,OX40N,NTT,DURABLES
48,KANSAINER,299.63,340.0,-67.31,H-SC,10.87,228420.0,-41247.0,77571.0,1.48,-15.30,33.96,13.47,143.0,-0.53,1.70,14.33,XY24,NTT,PAINTS
19,CERA,8421.60,8422.0,-12.41,X-SC,48.91,90480.0,-19001.0,19010.0,-1.30,-17.36,21.01,0.00,64.0,-1.00,0.67,36.94,OX40N,NTT,CERAMICS


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TCS,3794.03,4998.00,-17.12,X-LC,7.61,269997.0,-25937.0,119852.0,1.16,-8.76,44.39,31.73,1.0,-0.22,2.01,8.07,X40,BTT,IT
43,INFY,1461.46,2275.00,-10.98,X-LC,18.48,285594.0,23993.0,121634.0,1.44,9.17,42.59,55.67,2.0,0.20,2.13,15.84,X40,BTT,IT
52,LTIM,5564.16,7197.33,0.41,X-LC,61.96,193392.0,-6918.0,65715.0,2.17,-3.45,33.98,29.35,3.0,-0.11,1.44,33.61,X200,ATH,IT
7,ASIANPAINT,2961.56,4250.00,-23.91,X-LC,6.52,151232.0,-50154.0,137772.0,-0.15,-24.90,91.10,43.51,5.0,-0.36,1.13,5.27,X40,BTT,PAINTS
35,HINDUNILVR,2413.81,2723.00,-14.69,X-LC,25.00,267938.0,-2409.0,37029.0,-0.04,-0.89,13.82,12.81,6.0,-0.07,2.00,10.89,XY25,NTT,FMCG


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,UNIONBANK,123.87,163.0,-0.31,M-LC,92.39,173961.0,33121.0,11377.0,-1.54,23.52,6.54,31.59,88.0,2.91,1.30,51.11,XY24,NTT,BANKS
4,ADANIPORTS,1103.69,1583.0,4.69,M-LC,58.70,105037.0,26675.0,7353.0,0.84,34.04,7.00,43.43,77.0,3.63,0.78,40.49,XY24,NTT,MISC
51,LICI,785.05,983.0,-4.34,H-LC,84.78,235377.0,40685.0,8403.0,-1.43,20.90,3.57,25.21,34.0,4.84,1.75,30.16,XY25,NTT,INSURANCE


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SAIL,130.64,228.00,101.81,M-MC,72.83,229181.0,4219.0,163429.0,-0.55,1.88,71.31,74.53,183.0,0.03,1.71,33.06,XY24,BTT,STEEL
36,HINDZINC,514.80,744.74,43.14,M-LC,94.57,107200.0,4240.0,41744.0,2.10,4.12,38.94,44.67,68.0,0.10,0.80,38.34,X5K,ATH,METALS
18,CAMS,3643.00,5250.99,6.44,H-SC,86.96,119700.0,17696.0,27328.0,0.09,17.35,22.83,44.14,125.0,0.65,0.89,36.41,X40N,ATH,MISC
4,ADANIPORTS,1103.69,1583.00,4.69,M-LC,58.70,105037.0,26675.0,7353.0,0.84,34.04,7.00,43.43,77.0,3.63,0.78,40.49,XY24,NTT,MISC
33,HEROMOTOCO,4311.81,6039.03,0.48,H-MC,65.22,153678.0,2765.0,57691.0,0.95,1.83,37.54,40.06,101.0,0.05,1.14,25.70,AR,ATH,AUTO


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,116.91,190.00,-25.79,M-MC,81.52,195757.0,15949.0,96469.0,-1.57,8.87,49.28,62.52,168.0,0.17,1.46,40.87,XR,NTT,BANKS
90,WHIRLPOOL,1167.49,2270.00,-33.40,M-SC,42.39,194231.0,22610.0,139458.0,-2.47,13.17,71.80,94.43,191.0,0.16,1.45,43.35,XR,NTT,DURABLES
36,HINDZINC,514.80,744.74,43.14,M-LC,94.57,107200.0,4240.0,41744.0,2.10,4.12,38.94,44.67,68.0,0.10,0.80,38.34,X5K,ATH,METALS
34,HINDALCO,651.95,761.55,-11.72,M-LC,28.26,105720.0,1408.0,16133.0,1.53,1.35,15.26,16.81,71.0,0.09,0.79,17.61,X5K,ATH,METALS
91,WIPRO,243.46,420.00,-7.89,M-LC,16.30,157926.0,6981.0,102478.0,1.36,4.62,64.89,72.51,70.0,0.07,1.18,9.32,XR,NTT,IT


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,59.14,80.00,-53.72,L-SC,83.70,67162.0,-46387.0,86437.0,5.78,-40.85,128.70,35.27,260.0,-0.54,0.50,64.38,XR,NTT,HOTELS
5,ALKYLAMINE,2347.98,4567.52,-7.04,X-SC,89.13,86576.0,-14387.0,109830.0,0.05,-14.25,126.86,94.53,59.0,-0.13,0.64,28.36,SR,ATH,CHEMICALS
53,MASFIN,326.60,399.50,-19.60,H-SC,75.00,91350.0,-6630.0,28501.0,0.86,-6.77,31.20,22.32,138.0,-0.23,0.68,32.49,XR,ATH,FINANCE
4,ADANIPORTS,1103.69,1583.00,4.69,M-LC,58.70,105037.0,26675.0,7353.0,0.84,34.04,7.00,43.43,77.0,3.63,0.78,40.49,XY24,NTT,MISC
36,HINDZINC,514.80,744.74,43.14,M-LC,94.57,107200.0,4240.0,41744.0,2.10,4.12,38.94,44.67,68.0,0.10,0.80,38.34,X5K,ATH,METALS


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
86,VALIANTORG,512.64,838.00,-381.97,H-SC,100.00,119769.0,-13517.0,98115.0,4.99,-10.14,81.92,63.47,144.0,-0.14,0.89,100.76,XR,NTT,CHEMICALS
8,ASIANTILES,75.41,137.00,7016.67,M-SC,98.91,78098.0,-15712.0,92327.0,2.48,-16.75,118.22,81.67,235.0,-0.17,0.58,51.24,XR,NTT,CERAMICS
76,TANLA,917.30,1963.11,-32.99,H-SC,97.83,164169.0,-75246.0,348202.0,0.31,-31.43,212.10,114.01,127.0,-0.22,1.22,50.57,AR,ATH,IT
80,TITAGARH,1097.23,1548.00,-6.25,H-SC,96.74,164868.0,-27147.0,106027.0,-1.25,-14.14,64.31,41.08,152.0,-0.26,1.23,39.54,XY24,NTT,ENGINEERING
47,JPPOWER,18.73,26.20,-33.07,L-SC,95.65,136087.0,-5755.0,62328.0,1.35,-4.06,45.80,39.88,256.0,-0.09,1.01,37.91,XY24,NTT,POWER


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.42
1,25,43.09
2,50,71.82


In [21]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     33.28
XY25     15.14
XR       11.36
X40      10.77
OX40N     8.34
X40N      8.32
AR        7.86
X5K       2.35
X200      1.44
SR        1.15
Name: CurrAlloc%, dtype: float64

In [22]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    18.50
X-LC    17.88
M-SC    16.34
M-LC     9.98
H-MC     8.85
H-LC     7.74
X-MC     6.12
M-MC     5.80
L-SC     3.20
X-SC     2.74
L-LC     1.11
Name: CurrAlloc%, dtype: float64

In [23]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.95,-1.01,34.61
IT,10.99,-12.61,69.81
BANKS,10.24,-5.66,53.01
MISC,6.40,-25.86,75.07
FINANCE,6.35,-17.91,63.29
PAINTS,5.58,-16.59,39.57
INSURANCE,5.14,3.58,25.23
ELECTRICAL,5.12,-2.05,46.32
HEALTHCARE,4.63,-3.42,33.63


In [24]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2858721.0,26
XR,1189513.0,14
AR,1013977.0,9
X40,601480.0,8
XY25,497727.0,10
X40N,410271.0,8
OX40N,310759.0,11
SR,190269.0,2
X5K,87241.0,3


In [25]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1882630.0,18
M-SC,1682476.0,20
X-LC,811459.0,11
M-MC,565588.0,5
H-MC,538755.0,7
M-LC,375671.0,8
X-MC,322784.0,5
L-SC,299101.0,5
X-SC,263257.0,4


In [26]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,885340.0,6
M-SC,XY24,743649.0,6
H-SC,AR,535970.0,3
X-LC,X40,455697.0,4
M-SC,XR,418844.0,5
M-MC,XY24,383341.0,3
H-MC,XY24,224564.0,2
M-LC,XY24,203208.0,4
H-SC,XR,200253.0,3


In [27]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 33.0 seconds
